# The University of Melbourne, School of Computing and Information Systems
# COMP90049 Introduction to Machine Learning, 2020 Semester 1
-----
## Project 1: Understanding Student Success with Naive Bayes
-----
###### Student Name: Benjamin De Worsop
###### Python version: Python 3.6.8
###### Submission deadline: 11am, Wed 22 Apr 2019

This iPython notebook is a template which you will use for your Project 1 submission. 

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 


# Question 1 is found below
## 1) Part A
##### Explain the ‘naive’ assumption underlying Naive Bayes. (1) Why is it necessary? (2) Why can it be problematic? Link your discussion to the features of the students data set. [no programming required]

The Naive assumption in the Naive Bayes model is the assumption that each of the classes are all independent of each other. This is required to simply modelling probabilities of events. It allows you to compute the probabilities of combinations of events that haven't been seen before whilst still applying the logic behind Bayes rule. This can be seen in the example:

p(A+ | internet, absences) = p(internet, abences|A+)*p(A+)/p(internet, absences)

It is very difficult to find these probabilities. Instead, we can assume independence to get this equation:

p(A+ | internet, abences) = 
p(internet|A+)*p(abences|A+)*p(A+)/p(internet)*p(abences)

This is much easier to solve for because the variables are easier to measure as there is usually more data available on the occurrence of two overlapping variables (p(internet | A+)) than many overlapping variables. So, To be able to measure the probabilities of instances with previously unseen class combinations and to use more samples to guide probabilities, this assumption is necessary.

Unfortunately, predicting power is sacrificed when this assumption is made. The labelled probabilities of events are weaker approximations of their real probabilities. This is especially so when the assumption is less correct (the features are not very independent). In our class example, there are many values that we would assume are correlated like parents' jobs/education, extra paid class attendance and address which are all linked to wealth. Assuming these factors are probabilistically independent may be flawed, thus creating a more inaccurate result.


## 1) Part B


In [29]:
import pandas as pd
path = "student.csv"

# This function should open a data file in csv, and transform it into a usable format 
def load_data(path):
    data = pd.read_csv(path)
    return data

rawData = load_data(path)
print("done")

done


In [30]:
from sklearn.model_selection import train_test_split

# This function should split a data set into a training set and hold-out test set
def split_data(rawData):
    return train_test_split(rawData, test_size=0.1)

trainData, testData = split_data(rawData)
print("done")

done


In [31]:
import pandas as pd
featureFrames = None
GRADES = ["A+", "A","B","C","D","F"]


def make_grade_struct():
    #Create the grades df manually inserting them as column/row names
    featureFrames[len(list(rawData))-1] = pd.DataFrame(columns=GRADES, index=GRADES)

def make_data_structs():
    global featureFrames
    featureFrames = []
    paramNames = []

    #For each feature
    for feature in list(rawData):
        
        #Find all the names of the paramaters by iterating through all data
        for j in range(len(rawData)):
            if str(rawData[feature][j]) not in paramNames:
                paramNames.append(str(rawData[feature][j]))        
        
        #Make a dataframe to store frequency information with format:     
        #     A+    A    B    C    D    F
        # T  NaN  NaN  NaN  NaN  NaN  NaN
        # A  NaN  NaN  NaN  NaN  NaN  NaN
        
        df = pd.DataFrame(columns=GRADES, index=paramNames)
        featureFrames.append(df.copy())

        #Reset paramNames 
        paramNames = []
        
    #In case we don't see all the grade values, do this manually 
    make_grade_struct()    
    
def count_param_freq(data):
    #For every feature
    for i, feature in enumerate(list(data)):
        
        #Go and count the number of times each paramater resulted in what grade
        for j in range(len(data)):

            #Define the parameter feature, grade, and position in data struct
            param = str(data[feature].iloc[j])
            grade = data["Grade"].iloc[j]
            freqCount = featureFrames[i][grade][param]
        
            #Count the number of times that this parameter is seen
            if pd.isnull(freqCount):
                featureFrames[i][grade][param] = 1
            else:
                featureFrames[i][grade][param]+=1
                 

# This function should build a supervised NB model
def train(data):
    make_data_structs()
    count_param_freq(data)
    return


train(trainData)
print("done")

done


In [32]:
import operator
import random

eps = None
totTrainingRows = None


def find_probability(testGrade, testInst):

    #P(label|params) = const * p(param1|label) * p(param2|label) * ... * p(label)
    probability = 1
    
    #For every parameter, find the probability and update main prob
    #for every feature name
    for i, label in enumerate(rawData):
        #Don't use the "Grade" column for the instance for predictions
        #otherwise you're cheating!
        if i == len(list(rawData))-1:
            break
            

        # p(param1|label) = numberParam1|label / numberOfLabel
        #Get the actual paramater we're finding the probability of
        testParam = str(testInst[label])
        
        #Number of times this prameter was counted in training given the label
        try:
            #Try find the label, if you can't find it, or the value is null, probability *= eps
            paramFreq = featureFrames[i][testGrade][testParam]       
            if pd.isnull(paramFreq):
                probability*=eps
                continue
            #The number of times this grade has been counted in training
            gradeFreq = featureFrames[len(featureFrames)-1][testGrade][testGrade]            
       
            #Update probability
            probability *= (paramFreq/gradeFreq)
            
        except:
            probability*=eps
            continue

    try:
        probability *= gradeFreq/totTrainingRows
    except:
        probability = eps
        

    return probability

def predict_grade(instance):
    labelProbability = {"A+":0,"A":0,"B":0,"C":0,"D":0,"F":0}
    
    #For every potential label (A+, A, B, C, D, F), find P(label|params)
    for label in labelProbability.keys():
        
        labelProbability[label] = find_probability(label, instance)  
        
    return max(labelProbability.items(), key=operator.itemgetter(1))[0]


# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(data):
    global eps
    global totTrainingRows
    eps = 1/len(data)
    totTrainingRows = len(data)    
    
    for i, inst in enumerate(data.iterrows()):
        print("instance number {} is predicted to be = {}, actual grade = {}".format(i, predict_grade(inst[1]), inst[1]["Grade"]))
    


predict(testData)
print("done")


instance number 0 is predicted to be = D, actual grade = F
instance number 1 is predicted to be = C, actual grade = B
instance number 2 is predicted to be = D, actual grade = C
instance number 3 is predicted to be = C, actual grade = D
instance number 4 is predicted to be = C, actual grade = B
instance number 5 is predicted to be = F, actual grade = F
instance number 6 is predicted to be = F, actual grade = F
instance number 7 is predicted to be = C, actual grade = C
instance number 8 is predicted to be = B, actual grade = D
instance number 9 is predicted to be = C, actual grade = B
instance number 10 is predicted to be = C, actual grade = A
instance number 11 is predicted to be = A, actual grade = C
instance number 12 is predicted to be = A, actual grade = A+
instance number 13 is predicted to be = D, actual grade = D
instance number 14 is predicted to be = B, actual grade = B
instance number 15 is predicted to be = F, actual grade = F
instance number 16 is predicted to be = A, actual

In [33]:
accuracy = None


# This function should evaluate a set of predictions in terms of accuracy
def evaluate(data):
    count = 0
    global accuracy
    accuracy = {"success":0, "fail":0}
    print("evaluating")
    
    #Go through every row and count if the prediction was a success or fail
    for inst in data.iterrows():
        if(predict_grade(inst[1]) == inst[1]["Grade"]):
            accuracy["success"] += 1
        else:
            accuracy["fail"] += 1
    
    s = accuracy["success"]
    f = accuracy["fail"]
    print("accuracy is {}. ({} successes and {} fails)".format((s/(s+f)), s, f))
            
    return (s/(s+f))

evaluate(testData)
print("done")


evaluating
accuracy is 0.35384615384615387. (23 successes and 42 fails)
done


## 1) Part C

#### What accuracy does your classiﬁer achieve? Manually inspect a few instances for which your classiﬁer made correct predictions, and some for which it predicted incorrectly, and discuss any patterns you can ﬁnd.

From inspection, the accuracy of this model is around 30% for an 80-20 train-test data split. This is approximately twice as good as randomly guessing (\~16.6%) and about as good as guessing the most common class (\~30%). Observing the classifier output, it seems that this Naive Bayes model produces the better results more common classes within the dataset (Grade = C and D), and poorly with less common classes (Grade = A+). This may be due to instances with less common classes having less opportunity to have it's defining characteristics become more salient in the data. 

# Question 2

### A Closer Look at Evaluation

#### - A) You learnt in the lectures that precision, recall and f-1 measure can provide a more holistic and realistic picture of the classifier performance. (i) Explain the intuition behind accuracy, precision, recall, and F1-measure, (ii) contrast their utility, and (iii) discuss the difference between micro and macro averaging in the context of the data set. [no programming required]
#### - B) Compute precision, recall and f-1 measure of your model’s predictions on the test data set (1) separately for each class, and (2) as a single number using macro-averaging. Compare the results against your accuracy scores from Question 1. In the context of the student dataset, and your response to question 2a analyze the additional knowledge you gained about your classifier performance.



## 2) Part a

##### Intuition
Accuracy - This is the proportion of guesses we got right compared to all guesses. I.e. How many times was our prediction accurate? More formally:

(True positives + True Negatives)/(True or Flase positives or negatives)


Precision - For a class that we care to measure (maybe A+ students in this example), how often is our model correct. If we care about one label more than the others, we want to see how accurately we can predict that class. I.e. When our model detects an interesting class, is it right? More formally:

(True positives)/(True or False positives)


Recall - How good is our model at detecting our desired class. More formally:

(True positives)/(True positives + False Negatives)


F1 measure - A combined measure of precision and recall using the harmonic mean. This is important because precision and recall are usually inversely correlated (models with higher recall have lower precisino and vice versa). It shows how good your classifier at balancing precision and recall. It is defined:

(2 * Precision*Recall)/(Precision + Recall)


##### Compare/Contrast:
The value of precision and recall are based on your desire for type 1 vs type 2 error.

If you are a doctor and your model is determining if a patient has cancer, you don't want it to tell patients who have cancer, that they dont have cancer (false negative). Better safe than sorry. This is where you don't mind false positives, so you can have a low precision, but you need a high recall. 

This is opposite if you don't care about letting false negatives pass, but want to make sure those that you classify as part of an interesting class is important. An example of this is an employer who wants close to 100% of the candidates they interview to be high quality, but doesn't mind rejecting some potentially great candidates to make this happen.

F1 is important when you want a balance of these 2, and want to overall maximise your performance without having preference for type 1 or type 2 error.


##### Micro vs macro averaging
Macro averaging uses the precision and recall scores from each class and takes the average to produce one single score.

Micro averaging first aggregates all of the TP, FP and FN results from all the classes, and then adds, and divides these results by eachother according to the precision or recall equation respectively. E.g.

micro_precision = sum_TP/(sum_TP + sum_FP)


## 2) Part b


In [39]:
#First evaluate all the predicitons into TP, TN, FN and FP for each class

#Setup dataframes for storing the TP, TN, FN and FP results
labelAccuracy = {"A+":None,"A":None,"B":None,"C":None,"D":None,"F":None}

#Initialises dataframe
for label in labelAccuracy.keys():
    labelAccuracy[label] = pd.DataFrame(0, index=["positive", "negative"], columns=["true", "false"]) 

#Safely adds one to the right column of dataframe
def add_one(label, truFal, posNeg):
    if pd.isnull(labelAccuracy[label][truFal][posNeg]):
        labelAccuracy[label][truFal][posNeg] = 1
    else:
        labelAccuracy[label][truFal][posNeg] += 1

#Evaluate and count a particular type of error       
def count_error_type(label, predictedGrade, actualGrade):
    if(predictedGrade == actualGrade and predictedGrade == label):
        add_one(label, "true", "positive")
    elif(label != predictedGrade and label != actualGrade):
        add_one(label, "true", "negative")
    elif(label == predictedGrade and label != actualGrade):
        add_one(label, "false", "positive")
    elif(label != predictedGrade and label == actualGrade):
        add_one(label, "false", "negative")
    
# This function should evaluate a set of predictions in terms of accuracy
def evaluate_F1(data):
    #For each instance
    for inst in data.iterrows():
        
        predictedGrade = predict_grade(inst[1])
        actualGrade = inst[1]["Grade"]
        
        #Go through all the labels and find if it evaluated a TP, TN, FN or FP
        for label in labelAccuracy.keys():
            #If interesting key is correct, label as correct
            count_error_type(label, predictedGrade, actualGrade)
    return

eps = 1e-30

#Returns 0 if illegal division
def safe_divide(num, denom):
    if (denom < eps):
        return 0
    else:
        return num/denom
    
#Finds the average of a value found in a particular list location
def ave(lst, index): 
    sum = 0
    count = 0
    for indList in lst:
        sum += indList[index]
        count += 1
    return sum/count        

def print_func():
    mac_ave = []
    
    #Go through every grade (a key of the labelAccuracy dict)
    # and calculate important values
    for key in labelAccuracy.keys():
        theKey = key
        la = labelAccuracy
        
        precision = safe_divide(la[key]["true"]["positive"],(la[key]["true"]["positive"] + la[key]["false"]["positive"]))
        recall = safe_divide(la[key]["true"]["positive"],(la[key]["true"]["positive"] + la[key]["false"]["negative"]))
        f1 = safe_divide((2*precision*recall),(precision + recall))
        mac_ave.append([key, precision, recall, f1].copy())
      
        print("For the label {}, precision = {:.3f}, recall = {:.3f}, f-1 = {:.3f}".format(key, precision, recall, f1))
    print("macro averaging scores: precision = {:.3f}, recall = {:.3f}, f1 = {:.3f},".format(ave(mac_ave, 1),ave(mac_ave,2),ave(mac_ave,3)))
        
 

evaluate_F1(testData)
print_func()
print("done")

For the label A+, precision = 1.000, recall = 0.500, f-1 = 0.667
For the label A, precision = 0.333, recall = 0.364, f-1 = 0.348
For the label B, precision = 0.333, recall = 0.231, f-1 = 0.273
For the label C, precision = 0.176, recall = 0.231, f-1 = 0.200
For the label D, precision = 0.381, recall = 0.471, f-1 = 0.421
For the label F, precision = 0.600, recall = 0.333, f-1 = 0.429
macro averaging scores: precision = 0.471, recall = 0.355, f1 = 0.389,
done



## 2) Part b continued

#### Compare the results against your accuracy scores from Question 1. In the context of the student dataset, and your response to question 2a analyze the additional knowledge you gained about your classiﬁer performance.


The data gathered from part b supports the hypothesis created in Question 1 - that less common classes were predicted less accurately than more common classes around 20-30%. While the D and F classes can be predicted up to 40-50% of the time on a similar 80-20 data split, there are close-to or exactly 0 correct predictions of the A+ class depending on the random sampling. 

The model sometimes performs much better in recall for common classes like "C", "D" and "F ". This suggests a high number of false negatives were made for these classes. The prevelance of these classes suggest that this conservative guessing may be due to overfitting of their class traits. If the model has a very clear idea of what the "C", "D" and "F" classes looks like, it may be conservative with its guesses. A more balanced data set may help improve the accuracy of this classifier.



# Question 3: 
### Training Strategies 

#### There are other evaluation strategies, which tend to be preferred over the hold-out strategy you implemented in Question 1.

#### - A) Select one such strategy, (i) describe how it works, and (ii) explain why it is preferable over hold-out evaluation. [no programming required]

#### - B) Implement your chosen strategy from Question 3a, and report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.



## 3) Part a - Cross Validation


Cross validation separates the dataset into n chunks and iteratively uses each chunk as testing data, with the rest as training data. E.g. if n=3, on iteration 1: chunk 1 = test data, chunk 2 and 3 are used as training. Then, on iteration 2: chunk 2 = test, and chunk 1 and 3 are used for training. This process is repeated till every chunk is used as a test.

This is more effective as the hold-out strategy because it uses all of the data present in the set, unlike the holdout that uses one selection for testing and one selection for training. This makes the assessment more reflective of its overall performance using all the data.

By design, each run uses lots of data for training. As n increases, the model can be trained with more data on each iteration, making iterations more accurate. Having the model be the most informed during training creates the best proxy of how it would perform in real life.

Increasing the number of times the data is run gives a more accurate representation of the model's performance too. Outlier data doesn't affect the model as much because the final accuracy results aggregate all the previous results. The hold-out strategy is affected by this because it is only run once with possibly outlier-containing data.

It is also repeatable - giving flexibility to researchers to better understand how their model could be improved. 



## 3) Part b - Implementation of Cross Validation


In [40]:
n = 3
chunkSize = int(len(rawData)/n)
accuracyLog = []

#For each chunk, spit the data into testing and training data
for i in range(n):
    startIndex = i*chunkSize
    endIndex = startIndex + chunkSize
    
    #Deals with ugly divisors len(rawData)/chunkSize
    if i == n-1:
        endIndex = len(rawData)-1
    
    #Split the train and test data
    tempTrainDF = rawData.drop(rawData.loc[startIndex:endIndex]\
                               .index, inplace=False)
    tempTestDF = rawData.loc[startIndex:endIndex]

    #Train and evaluate results
    print("Iteration number {}...".format(i+1))
    train(tempTrainDF)
    accuracyLog.append(evaluate(tempTestDF))
    print()
    

print("average value = {}".format((sum(accuracyLog)/len(accuracyLog))))
print("done")


Iteration number 1...
evaluating
accuracy is 0.3317972350230415. (72 successes and 145 fails)

Iteration number 2...
evaluating
accuracy is 0.35023041474654376. (76 successes and 141 fails)

Iteration number 3...
evaluating
accuracy is 0.3640552995391705. (79 successes and 138 fails)

average value = 0.3486943164362519
done



## 3) Part b - Continued

#### Report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.

Unfortunately, the benefits of cross-validation don't seem to be apparent in this example. After experimenting with train-test separation values manually, an interesting guessing pattern emerged. The accuracy tends to be similar, even with different training-test splits.

The model, trained with low amounts of data, guessed primarily "D" and "F", the most popular label. This produced a similar accuracy of around 30%. As the amount of training data increased, it began predicting other classes. Although, even though it improved accuracy of less common classes, it reduced performance in more common classes almost linearly. This led to the overall accuracy remaining about the same. So, there was a 28-37% accuracy with most statistically significant splits (at a train-test 99-1 split, the results are not reliable because the law of large numbers don't apply and the results are too volatile).

This behaviour can largely be explained by the disproportionate spread of classes in the data. Ridding this bias would reduce the current model's higher accuracy at low training volumes due to the successful guess-the-most-popular-class strategy used. It may also make the differences between the classes more salient, creating more accurate classifiers when provided more training data. Other effects may be the low correlation between the classes and high-low performing student outcomes and/or high levels of dependence between the classes
